This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [1]:
# Lib imports
import os
import tensorflow as tf
# Use the ImageDataGenerator from the tensorflow runtime to avoid static import resolution issues
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator
# Access keras submodules via the tf object to avoid static import resolution issues
layers = tf.keras.layers
models = tf.keras.models
import numpy as np

In [2]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "./train"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "./test"    # e.g. './muffin-vs-chihuahua/test'

In [3]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [4]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [5]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Apply L2 regularization and stronger dropout (SpatialDropout2D for conv blocks)
l2_reg = tf.keras.regularizers.l2(1e-4)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2_reg, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.SpatialDropout2D(0.2),

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2_reg),
    layers.MaxPooling2D(2, 2),
    layers.SpatialDropout2D(0.2),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2_reg),
    layers.MaxPooling2D(2, 2),
    layers.SpatialDropout2D(0.2),

    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=l2_reg),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

c:\Users\ledes\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 32s 264ms/step - accuracy: 0.5993 - loss: 0.6922 - val_accuracy: 0.7672 - val_loss: 0.5301
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 32s 264ms/step - accuracy: 0.5993 - loss: 0.6922 - val_accuracy: 0.7672 - val_loss: 0.5301
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 31s 263ms/step - accuracy: 0.7606 - loss: 0.5369 - val_accuracy: 0.8317 - val_loss: 0.4255
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 31s 263ms/step - accuracy: 0.7606 - loss: 0.5369 - val_accuracy: 0.8317 - val_loss: 0.4255
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 32s 270ms/step - accuracy: 0.8017 - loss: 0.4787 - val_accuracy: 0.8635 - val_loss: 0.3610
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 32s 270ms/step - accuracy: 0.8017 - loss: 0.4787 - val_accuracy: 0.8635 - val_loss: 0.3610
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 33s 274ms/step - accuracy: 0.8049 - loss: 0.4777 - val_accuracy: 0.8772 - val_loss: 0.3619
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 33s 274ms/step - accuracy: 0.8049 - loss: 0

In [8]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 145ms/step - accuracy: 0.8733 - loss: 0.3375
Test Accuracy: 0.8733108043670654
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 145ms/step - accuracy: 0.8733 - loss: 0.3375
Test Accuracy: 0.8733108043670654


In [9]:
# SAVE THE MODEL
model.save('muffin_vs_chihuahua_cnn.h5')

In [10]:
# SIMPLE INFERENCE SCRIPT
# Use the tensorflow runtime object 'tf' already imported in the notebook
# to access the preprocessing utilities to avoid static import resolution issues.
image = tf.keras.preprocessing.image

def predict_image(img_path, model_path='muffin_vs_chihuahua_cnn.h5'):
    model = tf.keras.models.load_model(model_path, compile=False)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    print(f"Prediction: {label} (confidence: {pred:.2f})")


In [11]:
# Example usage:
predict_image("./predict/predict_1.jpg")
predict_image("./predict/predict_4.jpeg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Prediction: Chihuahua (confidence: 1.00)
Prediction: Chihuahua (confidence: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Prediction: Chihuahua (confidence: 0.98)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Prediction: Chihuahua (confidence: 0.98)
